In [35]:
from langgraph.graph import StateGraph
from typing import TypedDict
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.llms import Ollama

llm = Ollama(model="qwen2.5:latest")

def get_content(message):
    return message.content if hasattr(message, "content") else message

In [36]:
class ChainState(TypedDict):
	job_description: str
	resume_text: str
	job_description_summary: str
	resume_summary: str
	cover_letter: str

In [37]:
def generate_job_description_summary(state: ChainState) -> ChainState:
	prompt = """
		You are an expert job description analyst.
    Summarize the following job description in 4-6 bullet points.
    Focus on:
    - Core responsibilities
    - Required hard skills and tools
    - Required years of experience and education
    - Nice-to-have skills
    - Company culture or values mentioned
		
		Job Description:
    {state['job_description']}
    
    Return only the bullet points, no extra text.
	"""
  
	response = llm.invoke(prompt)

	return {
		**state,
		"job_description_summary": get_content(response)
	}

In [38]:
def generate_resume_summary(state: ChainState) -> ChainState:
	prompt = f""" 
		You are a senior career coach and professional resume writer.

    Using ONLY the actual experience and skills present in the candidate's resume below,
    write a powerful 4-6 sentence professional summary (the paragraph that goes at the top of a resume)
    that is perfectly tailored to the job description.

    Rules:
    - Do not invent or exaggerate experience that is not in the resume.
    - Mirror the language and keywords from the job description summary when possible.
    - Highlight the most relevant achievements and skills first.

    Resume (full text extracted from PDF):
    {state['resume_text']}

    Job Description Summary:
    {state['job_description_summary']}

    Output only the professional summary paragraph(s).
	"""

	response = llm.invoke(prompt)

	return { 
		**state,
		"resume_summary": get_content(response)
	}

In [39]:
def generate_cover_letter(state: ChainState) -> ChainState:
	prompt = f""" 
		You are an expert cover letter writer.

    Write a compelling, professional cover letter (max 400 words) using:
    - The candidate's actual background from their resume
    - The tailored resume summary you just created
    - The job description summary

    Structure:
    1. Strong opening paragraph explaining why they're excited and a perfect fit
    2. 2-3 paragraphs connecting specific experiences/accomplishments to the job requirements
    3. Closing paragraph with enthusiasm and call to action

    Resume text (for reference - use real achievements only):
    {state['resume_text']}

    Tailored Professional Summary:
    {state['resume_summary']}

    Job Description Summary:
    {state['job_description_summary']}

    Write in first person, confident but not arrogant tone.
	"""

	response = llm.invoke(prompt)

	return {
		**state,
		"cover_letter": get_content(response)
	}

In [40]:
workflow = StateGraph(ChainState)
workflow.add_node("generate_job_description_summary", generate_job_description_summary)
workflow.add_node("generate_resume_summary", generate_resume_summary)
workflow.add_node("generate_cover_letter", generate_cover_letter)

workflow.set_entry_point("generate_job_description_summary")
workflow.add_edge("generate_job_description_summary", "generate_resume_summary")
workflow.add_edge("generate_resume_summary", "generate_cover_letter")
workflow.set_finish_point("generate_cover_letter")

app = workflow.compile()

In [41]:
loader = PyPDFLoader("Leung_Yu_Lap_Resume.pdf")
resume_pages = loader.load()
resume_text = "\n".join([page.page_content for page in resume_pages])

job_description = """
iGears Technology Limited
Winter internship- AI & Software Engineering (STEM)
Design, develop, and maintain systems and websites with a focus on efficiency, scalability, and user-cantered design
Gather and analyse user requirements to support informed technical decisions
Develop system designs and write technical specifications
Conduct cross-browser testing to ensure consistent functionality
Collaborate with management and team members to meet project goals and timelines
Explore and implement AI solutions to enhance application functionality and user experience.
Stay updated on AI trends and technologies relevant to our projects.
Qualification - Computer Science (Bachelor Degree)
Experience - • Experience with modern front-end frameworks like React.js is highly desirable
Experience - • Experience in mobile development or AI technologies is beneficial
Skill - • Strong knowledge of PHP, HTML, HTML5, CSS, JavaScript, and SQL 
Skill - • Familiarity with open-source CMS platforms such as WordPress or Drupal is a plus
Skill - • Curiosity and enthusiasm for exploring new web and AI technologies, with a proactive learning mindset 
"""

result = app.invoke({
  "job_description": job_description,
	"resume_text": resume_text,
})

print(result["cover_letter"])

[Your Name]  
[Your Address]  
[City, State, ZIP Code]  
[Email Address]  
[Phone Number]  
[Date]

Hiring Manager, [Company's Name]  
[Company's Address]  
[City, State, ZIP Code]

Dear Hiring Manager,

I am writing to express my enthusiastic interest in the Project Manager position at [Company's Name], as advertised. With a strong foundation in Applied Computing and a passion for leveraging AI-assisted coding to enhance software development, I believe I can contribute significantly to your team’s success.

As a Year 4 Computer Science student from Hang Seng University of Hong Kong, I have honed my skills in project management, software engineering, and machine learning through diverse projects. My role as the Team Leader for the Rasa Travel Assistant Project allowed me to lead full-stack web development while integrating natural language processing (NLP) functionalities. This experience not only showcased my ability to manage complex projects but also demonstrated my proficiency in c